In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/ECE496/BVP')

Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
import numpy as np
import scipy.io as scio
import sys
import tensorflow as tf
import random

from sklearn.metrics import confusion_matrix
#from sklearn.metrics import classification_report
from sklearn.linear_model import RidgeClassifierCV
from sklearn.model_selection import train_test_split
from sktime.transformations.panel.rocket import MiniRocketMultivariate #MiniRocket, MiniRocketMultivariate

# **Local/Global Gesture ID Translation**

In [ ]:
room_dict = {'20181121':1,
'20181109':1,
'20181112':1,
'20181115':1,
'20181116':1,
'20181117':2,
'20181118':2,
'20181121':1,
'20181127':2,
'20181128':2,
'20181130':1,
'20181204':2,
'20181205':2,
'20181208':2,
'20181209':2,
'20181211':3
}

gesture_local_dict = {'20181121':{'1': 'Slide', '2': 'Draw-O(Horizontal)', '3': 'Draw-Zigzag(Horizontal)','4': 'Draw-N(Horizontal)', '5': 'Draw-Triangle(Horizontal)', '6': 'Draw-Rectangle(Horizontal)'},
'20181109':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide', '5': 'Draw-Zigzag(Vertical)', '6': 'Draw-N(Vertical)'},
'20181112':{'1': 'Draw-1', '2': 'Draw-2', '3': 'Draw-3', '4': 'Draw-4', '5': 'Draw-5', '6': 'Draw-6', '7': 'Draw-7', '8': 'Draw-8', '9': 'Draw-9', '10': 'Draw-0'},
'20181115':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Draw-O(Vertical)', '5': 'Draw-Zigzag(Vertical)', '6': 'Draw-N(Vertical)',},
'20181116':{'1': 'Draw-1', '2': 'Draw-2', '3': 'Draw-3', '4': 'Draw-4', '5': 'Draw-5', '6': 'Draw-6', '7': 'Draw-7', '8': 'Draw-8', '9': 'Draw-9', '10': 'Draw-0'},
'20181117':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Draw-O(Vertical)', '5': 'Draw-Zigzag(Vertical)', '6': 'Draw-N(Vertical)'},
'20181118':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Draw-O(Vertical)', '5': 'Draw-Zigzag(Vertical)', '6': 'Draw-N(Vertical)'},
'20181121':{'1': 'Slide', '2': 'Draw-O(Horizontal)', '3': 'Draw-Zigzag(Horizontal)', '4': 'Draw-N(Horizontal)', '5': 'Draw-Triangle(Horizontal)', '6': 'Draw-Rectangle(Horizontal)'},
'20181127':{'1': 'Slide', '2': 'Draw-O(Horizontal)', '3': 'Draw-Zigzag(Horizontal)', '4': 'Draw-N(Horizontal)', '5': 'Draw-Triangle(Horizontal)', '6': 'Draw-Rectangle(Horizontal)'},
'20181128':{'1':'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Draw-O(Horizontal)', '5': 'Draw-Zigzag(Horizontal)', '6': 'Draw-N(Horizontal)'},
'20181130':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide', '5': 'Draw-O(Horizontal)', '6': 'Draw-Zigzag(Horizontal)', '7': 'Draw-N(Horizontal)', '8': 'Draw-Triangle(Horizontal)', '9': 'Draw-Rectangle(Horizontal)'},
'20181204':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide','5': 'Draw-O(Horizontal)', '6': 'Draw-Zigzag(Horizontal)','7': 'Draw-N(Horizontal)', '8': 'Draw-Triangle(Horizontal)', '9': 'Draw-Rectangle(Horizontal)'},
'20181205':{'2':{'1': 'Draw-O(Horizontal)', '2': 'Draw-Zigzag(Horizontal)','3': 'Draw-N(Horizontal)', '4': 'Draw-Triangle(Horizontal)','5': 'Draw-Rectangle(Horizontal)'},'3':{'1': 'Slide', '2': 'Draw-O(Horizontal)', '3': 'Draw-Zigzag(Horizontal)', '4': 'Draw-N(Horizontal)', '5': 'Draw-Triangle(Horizontal)', '6': 'Draw-Rectangle(Horizontal)'}},
'20181208':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide'},
'20181209':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide', '5': 'Draw-O(Horizontal)', '6': 'Draw-Zigzag(Horizontal)'},
'20181211':{'1': 'Push&Pull', '2': 'Sweep', '3': 'Clap', '4': 'Slide', '5': 'Draw-O(Horizontal)', '6': 'Draw-Zigzag(Horizontal)'}
}

gesture_global_dict = {'Push&Pull': '1' ,
'Sweep': '2',
'Clap': '3',
'Slide': '4',
'Draw-N(Horizontal)':'5',
'Draw-O(Horizontal)':'6',
'Draw-Rectangle(Horizontal)':'7',
'Draw-Triangle(Horizontal)':'8',
'Draw-Zigzag(Horizontal)':'9',
'Draw-Zigzag(Vertical)':'10',
'Draw-N(Vertical)':'11',
'Draw-O(Vertical)':'12',
'Draw-1':'13',
'Draw-2':'14',
'Draw-3':'15',
'Draw-4':'16',
'Draw-5':'17',
'Draw-6':'18',
'Draw-7':'19',
'Draw-8':'20',
'Draw-9':'21',
'Draw-0':'22'}

# **Widar Data Functions**

In [ ]:
#Adapted from Widar3.0 code

def normalize_data(data_1):
    # data(ndarray)=>data_norm(ndarray): [20,20,T]=>[20,20,T]
    data_1_max = np.concatenate((data_1.max(axis=0),data_1.max(axis=1)),axis=0).max(axis=0)
    data_1_min = np.concatenate((data_1.min(axis=0),data_1.min(axis=1)),axis=0).min(axis=0)
    if (len(np.where((data_1_max - data_1_min) == 0)[0]) > 0):
        return data_1
    data_1_max_rep = np.tile(data_1_max,(data_1.shape[0],data_1.shape[1],1))
    data_1_min_rep = np.tile(data_1_min,(data_1.shape[0],data_1.shape[1],1))
    data_1_norm = (data_1 - data_1_min_rep) / (data_1_max_rep - data_1_min_rep)
    return  data_1_norm

def zero_padding(data, T_MAX):
    # data(list)=>data_pad(ndarray): [20,20,T1/T2/...]=>[20,20,T_MAX]
    data_pad = []
    for i in range(len(data)):
        t = np.array(data[i]).shape[2]
        data_pad.append(np.pad(data[i], ((0,0),(0,0),(T_MAX - t,0)), 'constant', constant_values = 0).tolist())
    return np.array(data_pad)

def load_data(path_to_data, motion_sel):
    global T_MAX
    data = []
    label = []
    T_MAX=0
    
    for data_root, data_dirs, data_files in os.walk(path_to_data):
        for data_file_name in data_files:
            file_path = os.path.join(data_root,data_file_name)
            try:
                data_1 = scio.loadmat(file_path)['velocity_spectrum_ro']
                label_1_local = int(data_file_name.split('-')[2])
                location = int(data_file_name.split('-')[3])
                orientation = int(data_file_name.split('-')[4])
                repetition = int(data_file_name.split('-')[5])

                folder_source= str(  str(file_path.split('/')[2]).split('-')[0]  )
                #print("Source Folder:" + folder_source)

                gesture_name= gesture_local_dict[folder_source][str(label_1_local)]
                label_1_global= int (gesture_global_dict[gesture_name])
                # Select Motion
                if (label_1_global not in motion_sel):
                    continue

                print('----------------------------------')
                print("File: ",file_path)
                print("Gesture name:", gesture_name)
                print("Local label:", label_1_local)
                print("Global label:", label_1_global)
                # Select Location
                # if (location not in [1,2,3,5]):
                #     continue

                # Select Orientation
                # if (orientation not in [1,2,4,5]):
                #     continue
                
                # Normalization
                data_normed_1 = normalize_data(data_1)
                # Update T_MAX
                if T_MAX < np.array(data_1).shape[2]:
                    T_MAX = np.array(data_1).shape[2]                
            except Exception:
                continue

            # Save List
            data.append(data_normed_1.tolist())
            #data.append(data_1.tolist())
            label.append(label_1_global)

    # Zero-padding
    data = zero_padding(data, T_MAX)
    print(data.shape)
    # Swap axes
    data = np.swapaxes(np.swapaxes(data, 1, 3), 2, 3)   # [N,20,20',T_MAX]=>[N,T_MAX,20,20']
    data = np.expand_dims(data, axis=-1)    # [N,T_MAX,20,20]=>[N,T_MAX,20,20,1]


    # Convert label to ndarray
    label = np.array(label)

    # data(ndarray): [N,T_MAX,20,20,1], label(ndarray): [N,N_MOTION]
    return data, label

# **Process and save data from each room into reusable numpy files**





In [ ]:
#Selected gestures:
ALL_MOTION = [1,2,3,4,6,9]
N_MOTION = len(ALL_MOTION)

In [ ]:
#Process and save Room 1 (only run first time, saves to OS/Drive as .npy)
data_dir1 = 'Data/ROOM1/'
data1, label1 = load_data(data_dir1, ALL_MOTION)
np.save("data1.npy",data1)
np.save("label1.npy",label1)
print('\nProcessed and saved dataset of ROOM 1, ' + str(label1.shape[0]) + ' samples, each sized ' + str(data1[0,:,:].shape) + '\n')

In [ ]:
#Process and save Room 2 (only run first time, saves to OS/Drive as .npy)
data_dir2 = 'Data/ROOM2/'
data2, label2 = load_data(data_dir2, ALL_MOTION)
np.save("data2.npy",data2)
np.save("label2.npy",label2)
print('\nProcessed and saved dataset of ROOM 2, ' + str(label2.shape[0]) + ' samples, each sized ' + str(data2[0,:,:].shape) + '\n')

In [ ]:
#Process and save Room 3 (only run first time, saves to OS/Drive as .npy)
data_dir3 = 'Data/ROOM3/'
data3, label3 = load_data(data_dir3, ALL_MOTION)
np.save("data3.npy",data3)
np.save("label3.npy",label3)
print('\nProcessed and saved dataset of ROOM 3, ' + str(label3.shape[0]) + ' samples, each sized ' + str(data3[0,:,:].shape) + '\n')

In [ ]:
#Combine and save all data from all rooms into a single file (only run first time, saves to OS/Drive as .npy)
data_all = np.concatenate((data1,data2),axis=0) 
label_all = np.concatenate((label1,label2),axis=0) 
data_all = np.concatenate((data_all,data3),axis=0) 
label_all = np.concatenate((label_all,label3),axis=0)

np.save("data_all.npy",data_all)
np.save("label_all.npy",label_all)
print('\nProcessed and saved dataset of all rooms, ' + str(label_all.shape[0]) + ' samples, each sized ' + str(data_all[0,:,:].shape) + '\n')

# **Load data from existing numpy files**

In [ ]:
#Load room 1 from OS/Drive save
data1 = np.load('data1.npy')
label1 = np.load('label1.npy')
print('\nLoaded dataset of Room 1, ' + str(label1.shape[0]) + ' samples, each sized ' + str(data1[0,:,:].shape) + '\n')


Loaded dataset of Room 1, 30717 samples, each sized (38, 20, 20, 1)



In [ ]:
#Load room 2 from OS/Drive save
data2 = np.load('data2.npy')
label2 = np.load('label2.npy')
print('\nLoaded dataset of Room 2, ' + str(label2.shape[0]) + ' samples, each sized ' + str(data2[0,:,:].shape) + '\n')


Loaded dataset of Room 2, 5300 samples, each sized (29, 20, 20, 1)



In [ ]:
#Load room 3 from OS/Drive save
data3 = np.load('data3.npy')
label3 = np.load('label3.npy')
print('\nLoaded dataset of Room 3, ' + str(label3.shape[0]) + ' samples, each sized ' + str(data3[0,:,:].shape) + '\n')


Loaded dataset of Room 3, 2995 samples, each sized (24, 20, 20, 1)



In [ ]:
#Load combined file from OS/Drive save
data_all= np.load('data_all.npy')
label_all= np.load('label_all.npy') 
print('\nLoaded dataset of all rooms, ' + str(label_all.shape[0]) + ' samples, each sized ' + str(data_all[0,:,:].shape) + '\n')


Loaded dataset of all rooms, 41963 samples, each sized (38, 20, 20, 1)



# **Training and Testing**

**Data configuration (select cell depending on experiment type)**

In [ ]:
#SELECT CROSS-DOMAIN (adjust individually)

#Train on:
data_train, label_train = data1, label1

#Test on:
data_test, label_test = data3, label3

In [ ]:
#SELECT IN-DOMAIN (adjust individually)
#Train and test on:
data = data2
label = label2
fraction_for_test = 0.1

#Optional: randomly select specific number of samples from data selected (e.g. limit to 3000 samples)
N= 0

if N!=0:
  index = random.sample(range(0, N), N-1)
  data=data[index]
  label=label[index]

#print(data.shape)
#print(label.shape)


#Split train and test randomly with fraction specified  (data, label, test_size=fraction_for_test)
[data_train, data_test, label_train, label_test] = train_test_split(data,label,test_size=fraction_for_test)

**Initialize MiniRocket instance, generate features, and train classifier**

In [ ]:
import time

data_train= np.array(data_train).astype('float32')
data_test= np.array(data_test).astype('float32')

#Flatten X and Y parameters (20x20) into single dimension (400)
data_train = data_train.reshape((data_train.shape[0],data_train.shape[1],data_train.shape[2]*data_train.shape[3]))
data_test = data_test.reshape((data_test.shape[0],data_test.shape[1],data_test.shape[2]*data_test.shape[3]))
data_train = data_train.swapaxes(1,2)
data_test = data_test.swapaxes(1,2)

#print("Input shape:", data_train.shape)

#Initialize a MiniRocket instance
minirocket_multi = MiniRocketMultivariate()


#Shape passed should be (N, 400, T_MAX) where N is number of samples, 400 is flattened X/Y of BVP, and T_MAX is the number of time recordings per sample
start_time_train = time.time()

minirocket_multi = minirocket_multi.fit(data_train)
X_train_transform = minirocket_multi.transform(data_train)
f_mean = X_train_transform.mean(0)
f_std = X_train_transform.std(0) + 1e-8
X_train_transform = (X_train_transform - f_mean) / f_std

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
classifier.fit(X_train_transform, label_train)

end_time_train = time.time()

**Evaluate classification accuracy and time complexity**

In [ ]:
#Overall accuracy
start_time_infer= time.time()

X_test_transform = minirocket_multi.transform(data_test)
X_test_transform = (X_test_transform - f_mean) / f_std
score = classifier.score(X_test_transform, label_test)

end_time_infer = time.time()

#Generate confusion matrix
label_pred = classifier.predict(X_test_transform)
print(set(label_pred.flatten()))
cm = confusion_matrix(label_test, label_pred)
print(cm)


print('\nAverage Accuracy: %f' %(score))

#Compute accuracy by gesture
for i in range(0,len(cm)):
  avg= cm[i][i]/ np.sum(cm[i])
  print('Gesture %d: %f'% (ALL_MOTION[i],avg) )

#Report time to train and infer
train_time = end_time_train - start_time_train
infer_time = end_time_infer - start_time_infer
print("\nTraining on %d samples took %f seconds" %(data_train.shape[0], train_time) )
print("Inference of %d samples took %f seconds" %(data_test.shape[0], infer_time) )